In [1]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [2]:
## Parameters and Global variables
district = "ksg"


## Load and Initialize Data

In [3]:
## Population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Number of regions
r = len(initDataDF.index)

## T-Matrix
tData = pd.read_csv("../output/" + district + "_tmatrix_results.csv")

## Sort distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)

In [4]:
tData.head(5)
# TODO fix the unnamed colum

,Unnamed: 0,iName,jName,Tij
0,0,Ajanur__Kasaragod,Ajanur__Kasaragod,48661.470000
1,1,Ajanur__Kasaragod,Badiyaduka__Kasaragod,1.071977
2,2,Ajanur__Kasaragod,Balal__Kasaragod,0.639348
3,3,Ajanur__Kasaragod,Bedadka__Kasaragod,5.770197
4,4,Ajanur__Kasaragod,Beloor__Kasaragod,0.254734


## Prepare T-Matrix

In [5]:

tMatrix = tData['Tij'].to_numpy().reshape(r,r)


## Function Definition

In [6]:
def getCij(i, j, tMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    cwSum = 0
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        Tjk = tMatrix[j][k]
        TlkSum = 0;

        for l in range(0, r-1):
            TlkSum = TlkSum + tMatrix[l][k]
        
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)

    return cwSum

In [7]:
getCij(0,1, tMatrix, initDataDF)

3.935769418313691e-05

## Compute C-Matrix

In [8]:
resultsDF = pd.DataFrame(columns=['iName', 'jName', 'Cij'], );
for i in range(0,r):
    for j in range(0,r):
        Cij = getCij(i,j,tMatrix, initDataDF)
        resultsDF = resultsDF.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Cij},ignore_index=True)
 

## Save Output

In [9]:
resultsDF.to_csv("../output/" + district + "_cmatrix_results.csv")

In [10]:
resultsDF.head(10)

,iName,jName,Cij
0,Ajanur__Kasaragod,Ajanur__Kasaragod,0.981142
1,Ajanur__Kasaragod,Badiyaduka__Kasaragod,0.000039
2,Ajanur__Kasaragod,Balal__Kasaragod,0.000031
3,Ajanur__Kasaragod,Bedadka__Kasaragod,0.000173
4,Ajanur__Kasaragod,Beloor__Kasaragod,0.000007
5,Ajanur__Kasaragod,Chemmanad__Kasaragod,0.000331
6,Ajanur__Kasaragod,Chengala__Kasaragod,0.000155
7,Ajanur__Kasaragod,Cheruvathur__Kasaragod,0.000142
8,Ajanur__Kasaragod,Delampadi__Kasaragod,0.000021
9,Ajanur__Kasaragod,East Eleri__Kasaragod,0.000028
